### Assisgment:
Create a simple assistant that uses any LLM and should be pydantic, when we ask about any product it should give you two information product Name, product details tentative price in USD (integer). use chat Prompt Template.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## Langsmith Tracking And Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.3-70b-versatile")

In [8]:
from pydantic import BaseModel,Field

class Product(BaseModel):
    product_name: str = Field(description="Name of the product")
    product_details: str = Field(description="A short description of the product")
    price_usd: int = Field(description="price of the product (tentative) in USD")

In [9]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(pydantic_object=Product)

In [10]:
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("you are a helpful assistant that provides structured product information.\n {format_instruction}"),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [20]:
messages = chat_prompt.format_messages(
    query = "tell me about iphone 16",
    format_instruction = parser.get_format_instructions()
)

response = model.invoke(messages)
product = parser.parse(response.content)

In [21]:
print(product)

{'product_name': 'iPhone 16', 'product_details': 'The iPhone 16 is a rumored upcoming smartphone from Apple, expected to feature a powerful A18 Bionic chip, improved cameras, and a sleek design.', 'price_usd': 999}
